In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [26]:
df = pd.read_csv('train.csv')
print(f"{np.round(df.isna().mean(), 4)*100} %")
df['family'] = df['SibSp'] + df['Parch']
df = df.drop(columns=['Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'])

df.head(3)

PassengerId     0.00
Survived        0.00
Pclass          0.00
Name            0.00
Sex             0.00
Age            19.87
SibSp           0.00
Parch           0.00
Ticket          0.00
Fare            0.00
Cabin          77.10
Embarked        0.22
dtype: float64 %


,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,family
0,1,0,3,male,22.0,7.2500,S,1
1,2,1,1,female,38.0,71.2833,C,1
2,3,1,3,female,26.0,7.9250,S,0


In [27]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked',
       'family'],
      dtype='object')

In [28]:
x = df.drop(columns='Survived')
y = df[['Survived']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ki = KNNImputer(n_neighbors=3, weights='distance')


num_cols = ['PassengerId', 'Pclass', 'Age', 'Fare', 'Embarked', 'family']
cat_cols = ['Sex', 'Embarked']


In [38]:
# encoding
cat_trf = ColumnTransformer([
    ('c1', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), cat_cols)
], remainder='passthrough')


pipe = Pipeline([
    ('p1', cat_trf),
    ('p2', StandardScaler()),
    ('p3', KNNImputer(n_neighbors=5))
])


pipe.fit(x_train)
# 2. NOW you can extract the names (because it's fitted)
ohe_cols = pipe.named_steps['p1'].named_transformers_['c1'].get_feature_names_out(cat_cols)
# 3. Get passthrough columns
passthrough_cols = [col for col in x_train.columns if col not in cat_cols]
# 4. Combine
all_cols = np.concatenate([ohe_cols, passthrough_cols])
# 5. Transform and Create DataFrame
x_train_trf = pd.DataFrame(pipe.transform(x_train), columns=all_cols)
x_train_trf


,Sex_male,Embarked_Q,Embarked_S,Embarked_nan,PassengerId,Pclass,Age,Fare,family
0,0.724310,-0.303355,0.596817,-0.053074,-0.453066,-1.614136,1.104489,-0.078684,-0.554666
1,0.724310,-0.303355,0.596817,-0.053074,1.113874,-0.400551,-0.448587,-0.377145,-0.554666
2,0.724310,-0.303355,0.596817,-0.053074,-0.254275,0.813034,0.172644,-0.474867,-0.554666
3,0.724310,-0.303355,0.596817,-0.053074,1.000836,0.813034,-0.241510,-0.476230,0.040096
4,-1.380624,-0.303355,0.596817,-0.053074,1.425702,0.813034,-1.622022,-0.025249,3.013909
...,...,...,...,...,...,...,...,...,...
707,-1.380624,-0.303355,0.596817,-0.053074,-1.330084,0.813034,-0.586638,-0.480162,-0.554666
708,0.724310,-0.303355,0.596817,-0.053074,-0.690835,-1.614136,1.532448,-0.030545,-0.554666
709,0.724310,-0.303355,0.596817,-0.053074,1.608902,0.813034,0.793874,-0.355804,0.634859
710,-1.380624,-0.303355,0.596817,-0.053074,-0.047689,-1.614136,-1.069817,1.683201,1.229621


In [ ]:
# scaling
